# RNN Based molucule generation 

Laurent Cetinsoy

In the hands-on we will train a character based RNN to generate smile molecules

We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model ! 

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

Import pandas and load the first 1000 lines

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("250k_smiles.csv")

Display the first rows of the dataframe

In [3]:
df.head()
#qed mesure de a quel point la molecule est probable

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


In [4]:
df["smiles"][3900]

'O=C(Nc1cccc(OCc2nnnn2C2CC2)c1)c1ccccc1\n'

## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [5]:
df["smiles"].str.len().max()

110

In [6]:
df["smiles"].str.len().argmax()

233038

In [7]:
df["smiles"][233038]

'C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@@]2(C)[C@H]1CC[C@H]2O[C@@H]1O[C@H](C(=O)[O-])[C@@H](O)[C@H](O)[C@H]1O\n'


Code a function **unic_characters(string)** which return the unic characters in a string


In [8]:
def unic_characters(string):
    return list(set(string))

In [9]:
unic_characters("fdgsf")

['s', 'd', 'f', 'g']

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [10]:
ucs = unic_characters(df["smiles"].str.cat())

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [21]:
def map_char_to_int(unic_chars):
    return dict([(uc, ord(uc)) for uc in unic_chars])
def map_int_to_char(unic_chars):
    return dict([(ord(uc), uc) for uc in unic_chars])

In [22]:
map_uc = map_char_to_int(ucs)
map_uc_rev = map_int_to_char(ucs)

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [23]:
map_uc

{'s': 115,
 'O': 79,
 '6': 54,
 '/': 47,
 '5': 53,
 'c': 99,
 '\n': 10,
 '(': 40,
 'F': 70,
 '-': 45,
 ']': 93,
 'o': 111,
 '+': 43,
 '#': 35,
 '3': 51,
 'l': 108,
 'S': 83,
 'r': 114,
 '7': 55,
 '1': 49,
 ')': 41,
 'P': 80,
 'N': 78,
 'C': 67,
 'I': 73,
 '\\': 92,
 '2': 50,
 'n': 110,
 '4': 52,
 '@': 64,
 'B': 66,
 'H': 72,
 '[': 91,
 '=': 61,
 '8': 56}

In [24]:
map_uc["C"]

67

In [25]:
map_uc_rev[67]

'C'

For each smile molecule add the ending token to it

In [26]:
unic_characters(df["smiles"].str[-1])

['\n']

In [27]:
df.isnull().values.any()

False

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [28]:
def build_X_and_y(string, i_char, seq_lenght):
    X = [string[i : i + seq_lenght] for i in i_char]
    y = [string[i + seq_lenght : i + seq_lenght + 1 ] for i in i_char]
    return X, y

In [29]:
build_X_and_y("etienneapasfaitlavaisselle",  [1,2,3], 4)

(['tien', 'ienn', 'enne'], ['n', 'e', 'a'])

In [30]:
build_X_and_y("az",  [1,2,3], 4)

(['z', '', ''], ['', '', ''])

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

In [31]:
from numpy import random

In [41]:
seq_length = 5

In [53]:
dataset = df["smiles"].apply(lambda stri : build_X_and_y(stri, random.randint(len(stri) // 2, size=6), seq_length))

In [151]:
X_train, y_train = zip(*dataset)

In [186]:
dataset


0         ([1ccc2, (CC(=, C)(C), C)(C), (CC(=, CC(C)], [...
1         ([[C@@H, c2cnc, (-c3n, C[C@@, 3nncn, [C@@H], [...
2         ([C(=O), (-c2c, c1ccc, #Cc1c, c(-c2, Cc1cc], [...
3         ([(=O)[, CCOC(, O)c2n, =O)c2, @H]1C, CCOC(], [...
4         ([c(Cl), C(=O), 1=C(S, O)Nc2, =O)Nc, 2)N=C], [...
                                ...                        
249450    ([(CNC(, [C@H], (C)CC, NC(=O, 1(C)C, (=O)C], [...
249451    ([cnc1C, cc(NC, ccc(N, ccnc1, c1C(=, C2CCN], [...
249452    ([)C(=O, (=O)C, )N(C), NC(=O, NC(=O, C(=O)], [...
249453    ([cc(C(, (OCC(, (OCC(, (=O)N, c2ccc, )Nc2c], [...
249454    ([(CC(c, ccc1), S(=O), c1)N1, c1ccc, 1)N1C], [...
Name: smiles, Length: 249455, dtype: object

Create numpy arrays from the lists

In [152]:
X_train, y_train

((['1ccc2', '(CC(=', 'C)(C)', 'C)(C)', '(CC(=', 'CC(C)'],
  ['[C@@H', 'c2cnc', '(-c3n', 'C[C@@', '3nncn', '[C@@H'],
  ['C(=O)', '(-c2c', 'c1ccc', '#Cc1c', 'c(-c2', 'Cc1cc'],
  ['(=O)[', 'CCOC(', 'O)c2n', '=O)c2', '@H]1C', 'CCOC('],
  ['c(Cl)', 'C(=O)', '1=C(S', 'O)Nc2', '=O)Nc', '2)N=C'],
  ['CC)[C', '](C)(', '](CC)', '[NH+]', '(CC)[', '@](C)'],
  ['cc(C(', 'Oc1cc', 'O)N(C', '@H](C', 'COc1c', 'ccc(C'],
  ['c[nH]', 'H]n1)', '1nc[n', 'C(Nc1', '(Nc1n', '1)c1c'],
  ['/c2cc', 'c(/C=', 'cn2)c', 'c(O)n', 'ccn2)', '2cc(B'],
  ['c2cc(', 'H]1CN', '=O)c2', 'c(Br)', '(=O)c', ')c2cc'],
  ['Oc1cc', 'c([C@', 'H]2C(', '=C(N)', '#N)=C', 'C)c(['],
  [')NC3C', ')C(=O', '=O)NC', '@H](C', 'O)NC3', ')NC3C'],
  ['O=C(N', '(N1CC', '(F)c2', '2c(F)', 'CCc2c', 'Cc2c('],
  ['C2(CC', 'cc1C(', 'ccccc', 'c1ccc', '1CCC2', 'c1C(='],
  ['(=O)C', 'CCc1c', 'c1cc(', 'CCCc1', 'C(=O)', 'c1cc('],
  ['CC(C)', 'CC(C)', 'C)Cc1', 'nc(SC', ']2CCC', ']2CCC'],
  ['(CNC(', 'cccc2', '(CNC(', '2NC(=', 'c(CNC', 'cc(CN'],
  ['CCCCC', 'C

In [153]:
X_train, y_train = np.array(X_train), np.array(y_train)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [154]:
X_train = X_train.reshape(X_train.shape[0] * X_train.shape[1])

In [155]:
X_train = np.array([list(st) for st in X_train])

In [156]:
X_train

array([['1', 'c', 'c', 'c', '2'],
       ['(', 'C', 'C', '(', '='],
       ['C', ')', '(', 'C', ')'],
       ...,
       ['c', '1', ')', 'N', '1'],
       ['c', '1', 'c', 'c', 'c'],
       ['1', ')', 'N', '1', 'C']], dtype='<U1')

Normalize X by dividing each values by the total number of unic characters

In [157]:
X_train.shape

(1496730, 5)

In [158]:
len_map = len(map_uc)

In [159]:
X_train = np.apply_along_axis(lambda l : [map_uc[li] / len_map for li in l], axis = 1, arr=X_train)

In [161]:
y_train = np.apply_along_axis(lambda l : [map_uc[li]/ len_map for li in l], axis = 0, arr=y_train)

In [162]:
y_train= y_train.reshape(-1)

In [163]:
X_train, y_train

(array([[1.4       , 2.82857143, 2.82857143, 2.82857143, 1.42857143],
        [1.14285714, 1.91428571, 1.91428571, 1.14285714, 1.74285714],
        [1.91428571, 1.17142857, 1.14285714, 1.91428571, 1.17142857],
        ...,
        [2.82857143, 1.4       , 1.17142857, 2.22857143, 1.4       ],
        [2.82857143, 1.4       , 2.82857143, 2.82857143, 2.82857143],
        [1.4       , 1.17142857, 2.22857143, 1.4       , 1.91428571]]),
 array([3.17142857, 2.25714286, 2.82857143, ..., 1.91428571, 2.82857143,
        1.91428571]))

In [164]:
X_train.shape, y_train.shape

((1496730, 5), (1496730,))

In [179]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [195]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten

In [198]:
model = Sequential(
    [
        LSTM(128, activation="tanh"),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(len_map, activation='softmax')
    ]
)

In [199]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=["accuracy"])
model.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
46773/46773 [==============================] - 245s 5ms/step - loss: 0.5287 - accuracy: 0.0026
Epoch 2/5
46773/46773 [==============================] - 243s 5ms/step - loss: 0.4592 - accuracy: 0.0029
Epoch 3/5
46773/46773 [==============================] - 246s 5ms/step - loss: 0.4463 - accuracy: 0.0029
Epoch 4/5
46773/46773 [==============================] - 244s 5ms/step - loss: 0.4393 - accuracy: 0.0028
Epoch 5/5
46773/46773 [==============================] - 246s 5ms/step - loss: 0.4345 - accuracy: 0.0029


In [183]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               66560     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  0 (unused)
_________________________________________________________________
dense_2 (Dense)              multiple                  0 (unused)
_________________________________________________________________
dense_3 (Dense)              multiple                  0 (unused)
Total params: 66,560
Trainable params: 66,560
Non-trainable params: 0
_________________________________________________________________


Compile the model with the appropriate loss function and the adam optimizer

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : predict the molecule solubility of your generated molecules 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better